In [851]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import requests
from bs4 import BeautifulSoup

# Load the data
DATA_PATH = ''

# Import tsv file, where each line is a movie and the metadata is separated by tabs and the first line is the header of columns : WikipediaID,WikidataID,PrequelID,SequelID
movie = pd.read_csv(DATA_PATH + 'sequels.csv', sep=',', header=0)

# adding the following categories to the movie dataframe ['WikipediaID', 'FreebaseID', 'MovieName', 'ReleaseDate', 'BoxOffice', 'Runtime', 'Languages', 'Countries', 'Genres']
movie['FreebaseID'] = ''
movie['MovieName'] = ''
movie['ReleaseDate'] = ''
movie['BoxOffice'] = ''
movie['Runtime'] = ''
movie['Languages'] = ''
movie['Countries'] = ''
movie['Genres'] = ''

In [852]:
# Creating a new dataframe with only the movies that have a sequel
movie_with_sequel = movie[movie['SequelID'].notnull()]

# For sequel, we get check if the movie is in the original dataframe. if it is not, we add it to a new dataframe
movie_with_sequel_not_in_original = movie_with_sequel[~movie_with_sequel['SequelID'].isin(movie['WikidataID'])]
print(movie_with_sequel_not_in_original.head)


<bound method NDFrame.head of        WikipediaID WikidataID  PrequelID    SequelID FreebaseID MovieName  \
17           77856    Q209170        NaN   Q24272881                        
41         9548445   Q4786598        NaN    Q3772186                        
135        1630107    Q602236   Q7744740    Q4753295                        
145       21878654   Q5643150  Q29565369   Q29565371                        
155       33495214    Q229557    Q230294    Q1354125                        
...            ...        ...        ...         ...        ...       ...   
81657       303933    Q683663        NaN  Q115932556                        
81695        54540    Q493755        NaN   Q64069446                        
81707     30863646   Q1152330        NaN    Q3860653                        
81723     28373912   Q6150472   Q3467727   Q12294915                        
81735     32468537    Q965863   Q2369836    Q4090322                        

      ReleaseDate BoxOffice Runtime Languages

In [853]:
def extract_name(html_extract):
    new_ID = []
    end_ID = False
    start_ID = False
    for i in range(len(html_extract)):
        if start_ID == False:
            if html_extract[i] == '"':
                start_ID = True
        elif start_ID == True:
            if end_ID == False:  
                if html_extract[i] == '"':
                    break
                else:
                    new_ID.append(html_extract[i])
    return new_ID

In [854]:
def extract_date(html_extract):
    new_date = []
    end_ID = False
    start_ID = False
    for i in range(len(html_extract)):
        if start_ID == False:
            if html_extract[i].isdigit():
                new_date.append(html_extract[i])
                start_ID = True
        elif start_ID == True:
            if end_ID == False:  
                if html_extract[i] == '\n':
                    break
                else:
                    new_date.append(html_extract[i])
    return new_date


In [855]:
def extract_revenue(html_extract):
    new_revenue = []
    end_ID = False
    start_ID = False
    for i in range(len(html_extract)):
        if start_ID == False:
            if html_extract[i].isdigit():
                new_revenue.append(html_extract[i])
                start_ID = True
        elif start_ID == True:
            if end_ID == False:  
                if html_extract[i] == '\n':
                    break
                else:
                    new_revenue.append(html_extract[i])
    return new_revenue

In [856]:
def extract_runtime(html_extract):
    new_runtime = []
    end_ID = False
    start_ID = False
    for i in range(len(html_extract)):
        if start_ID == False:
            if html_extract[i].isdigit():
                new_runtime.append(html_extract[i])
                start_ID = True
        elif start_ID == True:
            if end_ID == False:  
                if html_extract[i] == '\n':
                    break
                else:
                    new_runtime.append(html_extract[i])
    return new_runtime

In [857]:
def extract_languages(html_extract):
    new_languages = []
    end_ID = False
    start_ID = False
    text_ID = False
    for i in range(len(html_extract)):
        if start_ID == False:
            if html_extract[i] == '\n':
                start_ID = True
        elif start_ID == True:
            if end_ID == False:  
                if html_extract[i] == '\n':
                    break
                elif text_ID == False:
                    if html_extract[i].isalnum():
                        text_ID = True
                        new_languages.append(html_extract[i])
                else: 
                    new_languages.append(html_extract[i])
    return new_languages

In [858]:
def extract_genre(html_extract):
    new_genre = []
    end_ID = False
    start_ID = False
    text_ID = False
    for i in range(len(html_extract)):
        if start_ID == False:
            if html_extract[i] == '\n':
                start_ID = True
        elif start_ID == True:
            if end_ID == False:  
                if html_extract[i] == '\n':
                    break
                elif text_ID == False:
                    if html_extract[i].isalnum():
                        text_ID = True
                        new_genre.append(html_extract[i])
                else: 
                    new_genre.append(html_extract[i])
    return new_genre

In [859]:
def extract_sequel(html_extract):
    new_sequel = []
    end_ID = False
    start_ID = False
    for i in range(len(html_extract)):
        if start_ID == False:
            if html_extract[i] == 'Q':
                start_ID = True
                new_sequel.append(html_extract[i])
        elif start_ID == True:
            if end_ID == False:  
                if html_extract[i].isdigit() == True:
                    new_sequel.append(html_extract[i])
                else:
                    break
    return new_sequel

In [860]:
def scrape_metadata(df, i):
    metadata = ['', '', '', '', '', '', '']
    # test url 
    # url = 'https://www.wikidata.org/wiki/Q4786598'

    # specify the wikipedia page url
    url = "https://www.wikidata.org/wiki/" + str(df['SequelID'].iloc[i])

    # make a request to the url 
    response = requests.get(url)

    # parse the html content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # looking if the prequel and sequel index on the page exist
    movieName = soup.prettify().find('og:title') - 50
    releaseDate = soup.prettify().find('publication date')
    boxOffice = soup.prettify().find('box office')
    runtime = soup.prettify().find('duration')
    languages = soup.prettify().find('original language')
    genres = soup.prettify().find('genre')
    sequel = soup.prettify().find('followed by')

    print([movieName, releaseDate, boxOffice, runtime, languages, genres, sequel])


    if movieName != -1:
        movieName2 = soup.prettify()[movieName:].find('content="')
        movieName3 = soup.prettify()[movieName + movieName2 : movieName + movieName2 + 200]
        metadata[0] = "".join(extract_name(movieName3))
    else: 
        metadata[0] = "NaN"

    if releaseDate != -1:
        releaseDate2 = soup.prettify()[releaseDate:].find('December') or soup.prettify()[releaseDate:].find('November') or soup.prettify()[releaseDate:].find('October') or soup.prettify()[releaseDate:].find('September') or soup.prettify()[releaseDate:].find('August') or soup.prettify()[releaseDate:].find('July') or soup.prettify()[releaseDate:].find('June') or soup.prettify()[releaseDate:].find('May') or soup.prettify()[releaseDate:].find('April') or soup.prettify()[releaseDate:].find('March') or soup.prettify()[releaseDate:].find('February') or soup.prettify()[releaseDate:].find('January')
        releaseDate3 = soup.prettify()[releaseDate + releaseDate2 - 5 : releaseDate + releaseDate2 + 20]
        metadata[1] = "".join(extract_date(releaseDate3))
    else:
        metadata[1] = "NaN"

    if boxOffice != -1:
        boxOffice2 = soup.prettify()[boxOffice:].find('dollar') - 100
        boxOffice3 = soup.prettify()[boxOffice + boxOffice2 : boxOffice + boxOffice2 + 50]
        metadata[2] = "".join(extract_revenue(boxOffice3))
    else:
        metadata[2] = "NaN"

    if runtime != -1:
        runtime2 = soup.prettify()[runtime:].find('minute') or soup.prettify()[runtime:].find('hour')
        runtime3 = soup.prettify()[runtime + runtime2 - 100 : runtime + runtime2]
        metadata[3] = "".join(extract_runtime(runtime3))
    else:
        metadata[3] = "NaN"

    if languages != -1:
        languages2 = soup.prettify()[languages:].find('href="')
        languages3 = soup.prettify()[languages + languages2 : languages + languages2 + 100]
        metadata[4] = "".join(extract_languages(languages3))
    else:
        metadata[4] = "NaN"
        
    if genres != -1:
        genres2 = soup.prettify()[genres:].find('href="')
        genres3 = soup.prettify()[genres + genres2 : genres + genres2 + 100]
        metadata[5] = "".join(extract_genre(genres3))
    else:
        metadata[5] = "NaN"

    if sequel != -1:
        sequel_index_2 = soup.prettify()[sequel:].find('<a ')
        wikidata_id = soup.prettify()[sequel + sequel_index_2 : sequel + sequel_index_2 + 300]
        metadata[6] = "".join(extract_sequel(wikidata_id))
    else:
        metadata[6] = "NaN"

    return metadata

In [861]:
# for all sequels not in the original dataframe, we scrape the metadata and add it to the dataframe
metadata = ['', '', '', '', '', '', '']

# creating a dataframe for the new movies and their metadata
new_movies = pd.DataFrame(columns=['WikipediaID', 'WikidataID', 'PrequelID', 'SequelID', 'FreebaseID', 'MovieName', 'ReleaseDate', 'BoxOffice', 'Runtime', 'Languages', 'Countries', 'Genres'])

new_movies['WikidataID'] = movie_with_sequel_not_in_original['SequelID']
new_movies['PrequelID'] = movie_with_sequel_not_in_original['WikidataID']

for i in range(25):
    metadata = scrape_metadata(movie_with_sequel_not_in_original, i)
    print(metadata)

    # adding the metadata to the new dataframe   
    new_movies['SequelID'].iloc[i] = metadata[6]
    new_movies['MovieName'].iloc[i] = metadata[0]
    new_movies['ReleaseDate'].iloc[i] = metadata[1]
    new_movies['BoxOffice'].iloc[i] = metadata[2]
    new_movies['Runtime'].iloc[i] = metadata[3]
    new_movies['Languages'].iloc[i] = metadata[4]
    new_movies['Genres'].iloc[i] = metadata[5]

print(new_movies.head)
print(new_movies.shape)

#create new document with the results, where the name includes the start and end index
#online_scraping.to_csv('online_scraping_' + str(start) + '_' + str(end) + '.csv', sep=',', index=False, encoding='utf-8')

[5531, 61701, 340493, 310400, 59056, 39898, -1]
['Mary Poppins Returns', '19 December 2018', '346,800,000', '130', 'English', 'musical film', 'NaN']
[5495, -1, -1, -1, 41145, 27777, 25137]
["Archie's Weird Mysteries", 'NaN', 'NaN', 'NaN', 'English', 'teen sitcom', 'Q11721314']
[5500, -1, -1, -1, -1, 27040, 21171]
['And Do They Do/Zoo Caprices', 'NaN', 'NaN', 'NaN', 'NaN', 'contemporary classical music', 'Q2680386']
[5503, 40048, -1, 125835, 37403, -1, 29482]
['Porous Pockets/Choir Boys', '', 'NaN', '2047" title="Property:P2047">', 'English', 'NaN', 'Q29565373']
[5581, 36011, -1, 150436, 31640, 26398, 24718]
['1969 film by Harald Reinl', '', 'NaN', '95±0', 'German', 'comedy film', 'Q385581']
[5493, 80304, 298112, 266395, 75935, 44369, -1]
['RoboCop', '10 December 2017', '242,688,965', '117', 'English', 'action film', 'NaN']
[5512, 44351, -1, 20893, 39978, 32991, -1]
['Gertie on Tour', '', 'NaN', '2', 'English', 'silent film', 'NaN']
[5499, 40018, -1, -1, -1, 23985, 21349]
['Messages: Gr